In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import preprocess as prep
import os
import shutil

In [ ]:
# Calculate the corner point of the picture 
def corner_point(position):
    x1, y1, x2, y2 = position
    return int(min(x1, x2)), int(min(y1, y2)), int(max(x1, x2)), int(max(y1, y2))

In [ ]:
# Refresh the data. Return the selected region of data
def data_refresh(data, x_final, y_final, window_shape_value):
    data_refresh = data[y_final:y_final+window_shape_value, x_final:x_final+window_shape_value]
    return data_refresh

In [ ]:
# Save the data and asteroid position into a *.npz file
def save_data_negative(name, index, count, data, data_dir):
    dataPath = '{}\{}_{}'.format(data_dir, count, name)
    np.savez(dataPath, matrix1=data)
    print('Successfully save data for index:{} count:{} filename:{}'.format(index, count, name))

In [ ]:
# Plot the data and save into a *.jpg file
def plot_data_negative(name, count, data, pic_dir): 
    plt.imshow(data, cmap='gray', vmin=0, vmax=0.01)
    plt.colorbar()
    plt.savefig('{}\{}_{}.jpg'.format(pic_dir, count, name), dpi=300)
    plt.clf()

In [ ]:
# Find the posible negative pictures
def find_region(data, corner, window_shape_value, name, index, count, data_dir, pic_dir):
    x_min, y_min, x_max, y_max = corner
    
    x_final = []
    y_final = []
    
    if x_min > window_shape_value+500:
        x_final.append(random.randint(0, x_min - window_shape_value - 300))
    
    if (4000 - x_max) > window_shape_value+500:
        x_final.append(random.randint(x_max+300, 4000 - window_shape_value))
        
    if y_min > window_shape_value+500:
        y_final.append(random.randint(0, y_min - window_shape_value - 300))
        
    if (4000 - y_max) > window_shape_value+500:
        y_final.append(random.randint(y_max+300, 4000 - window_shape_value))
        
    if x_final == [] or y_final == []:
        return 0
    
    for i in x_final:
        for j in y_final:
            data_new = data_refresh(data, i, j, window_shape_value)
            save_data_negative(name, index, count, data_new, data_dir)
            plot_data_negative(name, count, data_new, pic_dir)

In [ ]:
# Wrap it up
def generate_negative_data(father_dir, pic_dir, data_dir, start_index, end_index, window_shape_value=1000):
    # Count the number of accepted negative picture
    count = 0
    
    # Iterate through the data
    for index in range(start_index, end_index):
        # Get the path and filename of the original data file (generated by download.py)
        path, name = prep.path_of_data(father_dir, index)
        print('Start preprocessing {}: {}'.format(index, name))
        
        # Load the *.npz file, and get the position of the asteroid trail
        data, position = prep.load_saved_data(path)
        
        # Convert the 'nan' value to 0 in the array
        data = np.nan_to_num(data)
        
        # Calculate the corner of the asteroid trail
        corner = corner_point(position)
        
        # Check if we can use this image to generate a negative picture.
        # If so, randomly pick two pictures with no asteroid trails.
        find_region(data, corner, window_shape_value, name, index, count, data_dir, pic_dir)
                

In [ ]:
# Randomly split the data set to train and validation. We choose a 80/20 split.
def random_split(dir_train, dir_val):
    fileList = os.listdir(dir_train)
    for file in fileList:
        train_file = os.path.join(dir_train, file)
        val_file = os.path.join(dir_val, file)
        if random.random() < 0.2:
            shutil.move(train_file, val_file)
            print('File move from {} to {}'.format(train_file, val_file))
        else:
            print('File does not move')

In [ ]:
# Main Function
if __name__ == '__main__':
    generate_negative_data('.\data', 
                           '.\pic_negative', 
                           '.\\preprocess\\train\\negative', 
                           start_index=0, end_index=30, 
                           window_shape_value=1000)
    random_split('.\\preprocess\\train\\negative', 
                 '.\\preprocess\\val\\negative')